In [4]:
import qutip
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import pickle
import time as timess
import math, cmath
import sys
import auxiliary_library as me

from IPython.display import display, Math, Latex
np.set_printoptions(threshold=1.e-9,linewidth=120,precision=3, formatter={"float":lambda x: str(.001*int(1000*x)) })

In [14]:
N = 3
spin_ops_list = me.one_body_spin_ops(N)

### Heisenberg Hamiltonian
chain_type = "XXZ"; Hamiltonian_paras = [.2, .15, .1, 1.]
H_H = me.Heisenberg_Hamiltonian(op_list = spin_ops_list, chain_type = chain_type,
                                size = N, Hamiltonian_paras = Hamiltonian_paras,
                                closed_bcs = True, visualization = False)

### Basis operators 

cl_ops, label_ops = me.classical_ops(H_H, N, spin_ops_list, False)
magnetization = sum(spin_ops_list[3][a] for a in range(len(spin_ops_list[3])))
neel_operator = sum((-1)**a * spin_ops_list[3][a] for a in range(len(spin_ops_list[3])))
identity_op = spin_ops_list[0][0]

label_ops.append("magnetization"); label_ops.append("Neel_op"); 
cl_ops[label_ops[6]] = magnetization; cl_ops[label_ops[7]] = neel_operator; 

print("Are all the operators Hermitian?:", me.basis_hermitian_check(cl_ops))
cl_ops = me.Hamiltonian_comm_check(H_H, cl_ops, None, True)

rho_ref = (-H_H - .5*(cl_ops["n_oc_op"] - identity_op)**2).expm()
rho_ref = rho_ref/rho_ref.tr()

basis = list(cl_ops.values())

Are all the operators Hermitian?: True
[H,  x_op ] = 0?:  False
[H,  p_op ] = 0?:  False
[H,  n_oc_op ] = 0?:  False
[H,  comm_xp ] = 0?:  False
[H,  corr_xp ] = 0?:  False
[H,  p_dot ] = 0?:  False
[H,  magnetization ] = 0?:  True
magnetization basis element deleted
[H,  Neel_op ] = 0?:  False


In [17]:
basis2 = me.base_orth(basis, rho_ref, me.HS_inner_prod_t)

In [54]:
rho0 = psi0 * psi0.dag()
rho_ref = (-H_H - .5*(cl_ops["n_oc_op"] - spin_ops_list[0][0])**2).expm()
rho_ref = rho_ref/rho_ref.tr()

op_basis = gs_basis(cl_ops, rho_ref, me.HS_inner_prod_t)

# initial conditions 

phi0 = [0., 0., 2., 2., .1, .0, -2] # for i in range(len(op_basis))]
rho0 = (-sum( f*op for f,op in zip(phi0, op_basis))).expm()
#phi0[0] = np.log(rho0.tr())
#rho0 = (-sum( f*op for f,op in zip(phi0, op_basis))).expm()

#print("valores medios iniciales:",{name:qutip.expect(op, rho0) for name, op in zip(["n_oc","x","p", "[x,p]/I", 
#                                                                         "I [H,p]"],[n_oc, x,p,comm, ac])})

H_ij_tensor = me.H_ij_matrix(H_H, op_basis, rho0, me.HS_inner_prod_t)
plt.plot(qutip.Qobj(HijT).eigenenergies().real)
plt.plot(sorted(qutip.Qobj(HijT).eigenenergies().imag))

SystemExit: rho0 is not a density op

In [ ]:
### Init state 
psi0 = ((1-.1j)*qutip.tensor([qutip.basis(2,1) for k in range(int(N/2)-1)]+
                    [qutip.basis(2,0)]+
                    [qutip.basis(2,1) for k in range(int(N/2+.5))])
        -
        (1+.1j)*qutip.tensor([qutip.basis(2,1) for k in range(int(N/2))]+
                    [qutip.basis(2,0)]+
                    [qutip.basis(2,1) for k in range(int(N/2+.5)-1)]))*np.sqrt(.5/1.01)

print("initial mean values:",{name:qutip.expect(op, psi0) for name, op in zip(list(cl_ops.keys()), list(cl_ops.values()))})